# Dimensions Publications CSV Merge (2000–2025) and Parquet Conversion

## Background and Objectives
This notebook merges yearly Dimensions CSV files for publications from 2000 to 2025 and converts the merged CSV to Parquet. The workflow follows a robust, reproducible process suitable for large-scale bibliometric datasets.

- Source directory: `data/raw/dimensions_cs`
- Files merged: `publications_2000.csv` … `publications_2025.csv`
- Column policy: union-of-columns; missing values are encoded as empty strings in CSV
- Output directory: `data/processed`
- Outputs: `dimension_merged.csv`, `dimension_merged.parquet`

The implementation emphasizes transparency, reproducibility, and performance using streaming I/O and DuckDB for efficient CSV→Parquet conversion.


## Methods Overview

The notebook is organized into the following stages:
1. Environment setup and path configuration
2. Input file validation (2000–2025)
3. Column inspection (union-of-columns policy)
4. Streaming merge to a single CSV (kept for audit)
5. CSV → Parquet conversion via DuckDB
6. Output validation and sampling
7. Reproducibility metadata (package versions, runtime)

We keep the intermediate CSV to enable downstream auditability and external tool usage.


In [1]:
# Environment setup
from pathlib import Path
import os
import csv
import time
from typing import List, Optional, Set, Iterable

import pandas as pd
import pyarrow as pa

# Ensure duckdb is available
try:
    import duckdb  # type: ignore
except Exception:
    import sys, subprocess
    subprocess.run([sys.executable, "-m", "pip", "install", "--quiet", "duckdb>=0.10.0"], check=True)
    import duckdb  # type: ignore

PROJECT_ROOT = Path(os.getcwd()).parent.parent
INPUT_DIR = PROJECT_ROOT / "data/raw/dimensions_cs"
OUTPUT_DIR = PROJECT_ROOT / "data/processed"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Inputs (2000–2025)
INPUT_FILES = [INPUT_DIR / f"publications_{year}.csv" for year in range(2000, 2026)]

# Outputs
OUTPUT_CSV = OUTPUT_DIR / "dimension_merged.csv"
OUTPUT_PARQUET = OUTPUT_DIR / "dimension_merged.parquet"

print("Project root:", PROJECT_ROOT)
print("Input dir:", INPUT_DIR)
print("Output dir:", OUTPUT_DIR)
print("Output CSV:", OUTPUT_CSV)
print("Output Parquet:", OUTPUT_PARQUET)


Project root: /Users/yann.jy/InvisibleResearch
Input dir: /Users/yann.jy/InvisibleResearch/data/raw/dimensions_cs
Output dir: /Users/yann.jy/InvisibleResearch/data/processed
Output CSV: /Users/yann.jy/InvisibleResearch/data/processed/dimension_merged.csv
Output Parquet: /Users/yann.jy/InvisibleResearch/data/processed/dimension_merged.parquet


## Validate Input Files
We require all yearly files to exist prior to merging. This ensures deterministic behavior and eases troubleshooting.


In [2]:
existing_files = [p for p in INPUT_FILES if p.exists()]
missing = [p for p in INPUT_FILES if not p.exists()]

if not existing_files:
    raise FileNotFoundError("No input files found under " + str(INPUT_DIR))

print(f"Existing files: {len(existing_files)}")
if missing:
    print("Missing files (skipped):")
    for m in missing[:10]:
        print(" -", m)
    if len(missing) > 10:
        print(f" ... (+{len(missing)-10} more)")

EXISTING_FILES = existing_files


Existing files: 26


## Column Inspection (Union-of-Columns)
We read only header lines to compute the union of columns and detect schema differences across years.


In [3]:
from typing import Tuple

def read_header(path: Path) -> List[str]:
    with path.open("r", encoding="utf-8-sig", errors="replace", newline="") as f:  # utf-8-sig strips BOM
        reader = csv.reader(
            f,
            delimiter=",",
            quotechar='"',
            doublequote=True,
            escapechar="\\",
        )
        header = next(reader)
        return [c.strip() for c in header]

union_cols: Set[str] = set()
all_same = True
baseline: Optional[List[str]] = None
differing_files: List[str] = []

for p in EXISTING_FILES:
    cols = read_header(p)
    union_cols.update(cols)
    if baseline is None:
        baseline = cols
    elif cols != baseline:
        all_same = False
        differing_files.append(str(p))

UNION_COLUMNS = sorted(list(union_cols))
print(f"Total unique columns: {len(UNION_COLUMNS)}")
print("All files share same columns:" if all_same else "Column differences detected.")
if differing_files:
    print("Examples of differing files (first 5):")
    for ex in differing_files[:5]:
        print(" -", ex)


Total unique columns: 76
Column differences detected.
Examples of differing files (first 5):
 - /Users/yann.jy/InvisibleResearch/data/raw/dimensions_cs/publications_2001.csv
 - /Users/yann.jy/InvisibleResearch/data/raw/dimensions_cs/publications_2002.csv
 - /Users/yann.jy/InvisibleResearch/data/raw/dimensions_cs/publications_2003.csv
 - /Users/yann.jy/InvisibleResearch/data/raw/dimensions_cs/publications_2004.csv
 - /Users/yann.jy/InvisibleResearch/data/raw/dimensions_cs/publications_2005.csv


## Streaming Merge to CSV
We stream through each yearly CSV, align rows to the union of columns, and write to a single CSV. Missing columns in a given year are written as empty strings. The intermediate CSV is kept for auditability and potential reuse.


In [4]:
def merge_to_csv(input_paths: List[Path], union_columns: List[str], output_csv_path: Path) -> int:
    output_csv_path.parent.mkdir(parents=True, exist_ok=True)
    total_rows = 0

    with output_csv_path.open("w", encoding="utf-8", errors="replace", newline="") as out_f:
        writer = csv.writer(out_f, delimiter=",", quotechar='"', lineterminator="\n")
        writer.writerow(union_columns)

        for idx, path in enumerate(input_paths, start=1):
            with path.open("r", encoding="utf-8-sig", errors="replace", newline="") as in_f:
                reader = csv.reader(
                    in_f,
                    delimiter=",",
                    quotechar='"',
                    doublequote=True,
                    escapechar="\\",
                )
                header = next(reader, None)
                if header is None:
                    continue
                header = [c.strip() for c in header]
                name_to_idx = {name: i for i, name in enumerate(header)}

                for row in reader:
                    out_row = []
                    row_len = len(row)
                    for col in union_columns:
                        idx_col = name_to_idx.get(col)
                        out_row.append(row[idx_col] if idx_col is not None and idx_col < row_len else "")
                    writer.writerow(out_row)
                    total_rows += 1

            if idx % 5 == 0:
                print(f"Processed {idx}/{len(input_paths)} files. Total rows so far: {total_rows:,}")

    return total_rows

# Increase CSV field size limit to accommodate long text fields
try:
    csv.field_size_limit(10_000_000)
except Exception:
    pass

rows_written = merge_to_csv(EXISTING_FILES, UNION_COLUMNS, OUTPUT_CSV)
print(f"Merged CSV rows: {rows_written:,}")


Processed 5/26 files. Total rows so far: 21,376
Processed 10/26 files. Total rows so far: 54,386
Processed 15/26 files. Total rows so far: 108,532
Processed 20/26 files. Total rows so far: 199,011
Processed 25/26 files. Total rows so far: 351,923
Merged CSV rows: 358,493


## CSV → Parquet Conversion via DuckDB
We convert the merged CSV to Parquet using DuckDB's COPY command with an explicit VARCHAR mapping to avoid implicit type inference pitfalls and ensure schema consistency.


In [5]:
# Build explicit column mapping for DuckDB as VARCHAR
columns_pairs = ", ".join([f"'{c}':'VARCHAR'" for c in UNION_COLUMNS])
columns_spec = "{" + columns_pairs + "}"

# Clean output parquet if exists
if OUTPUT_PARQUET.exists():
    OUTPUT_PARQUET.unlink()

con = duckdb.connect()
con.execute("PRAGMA threads=4")

safe_csv = str(OUTPUT_CSV).replace("'", "''")
safe_parquet = str(OUTPUT_PARQUET).replace("'", "''")

copy_sql = (
    f"COPY (SELECT * FROM read_csv('{safe_csv}', AUTO_DETECT=FALSE, HEADER=TRUE, "
    f"COLUMNS={columns_spec}, delim=',', quote='\"', escape='\"', strict_mode=FALSE, null_padding=TRUE, "
    f"maximum_line_size=20000000, parallel=FALSE)) TO '{safe_parquet}' (FORMAT 'parquet', COMPRESSION 'SNAPPY')"
)
print("Executing DuckDB COPY → Parquet ...")
con.execute(copy_sql)

# Count rows in Parquet
row_count = con.execute(f"SELECT COUNT(*) FROM read_parquet('{safe_parquet}')").fetchone()[0]
con.close()
print(f"Parquet rows: {row_count:,}")


Executing DuckDB COPY → Parquet ...
Parquet rows: 358,493


## Validation and Sampling
We validate data integrity beyond simple row counts: logical CSV row counting, `id` uniqueness, DOI normalization duplicates, year range/distribution, key field quality, optional exact-duplicate detection, and sampling.


In [8]:
import pyarrow.parquet as pq
import duckdb

# Count logical rows in CSV using csv.reader (handles multiline fields correctly)
print("Counting CSV logical rows (this may take a moment)...")
csv_rows = 0
with OUTPUT_CSV.open('r', encoding='utf-8-sig', errors='replace', newline='') as f:
    reader = csv.reader(f, delimiter=',', quotechar='"', doublequote=True, escapechar='\\')
    next(reader, None)  # skip header
    for _ in reader:
        csv_rows += 1
print(f"CSV logical rows: {csv_rows:,}")

# Read Parquet metadata and show schema
parquet_file = pq.ParquetFile(OUTPUT_PARQUET)
parquet_rows = parquet_file.metadata.num_rows
print(f"Parquet rows: {parquet_rows:,}")
print(f"Parquet columns: {len(parquet_file.schema)}")
print(f"Row groups: {parquet_file.num_row_groups}")

# Validate row count consistency
if csv_rows == parquet_rows:
    print(f"✅ Row count validated: CSV and Parquet both have {csv_rows:,} records")
else:
    print(f"⚠️ Row count mismatch: CSV={csv_rows:,}, Parquet={parquet_rows:,}, diff={abs(csv_rows-parquet_rows):,}")

# Build a DuckDB view on Parquet for further checks
con = duckdb.connect()
parquet_path = str(OUTPUT_PARQUET).replace("'", "''")
con.execute(f"CREATE OR REPLACE VIEW v AS SELECT * FROM read_parquet('{parquet_path}')")

cols = [r[0] for r in con.execute("DESCRIBE SELECT * FROM v").fetchall()]

def q(name: str) -> str:
    return '"' + name.replace('"','""') + '"'

# 1) id uniqueness (as requested)
if 'id' in cols:
    id_stats = con.execute(
        f"""
        SELECT 
          COUNT(*)::BIGINT AS total,
          COUNT(DISTINCT {q('id')})::BIGINT AS distinct_ids,
          SUM(CASE WHEN {q('id')} IS NULL OR {q('id')}='' THEN 1 ELSE 0 END)::BIGINT AS null_ids
        FROM v
        """
    ).fetch_df()
    total, distinct_ids, null_ids = int(id_stats['total'][0]), int(id_stats['distinct_ids'][0]), int(id_stats['null_ids'][0])
    print(f"\n[id] total={total:,}, distinct={distinct_ids:,}, nulls={null_ids:,}, unique={distinct_ids==total and null_ids==0}")

    # show any duplicated ids (top 10)
    if not (distinct_ids==total and null_ids==0):
        print("Examples of duplicated id:")
        print(con.execute(
            f"""
            SELECT {q('id')} AS id, COUNT(*) AS c
            FROM v
            GROUP BY 1
            HAVING COUNT(*) > 1
            ORDER BY c DESC
            LIMIT 10
            """
        ).fetch_df())
else:
    print("\n[id] column not found; skip uniqueness check.")

# 2) DOI normalization duplicates (if doi exists)
if 'doi' in cols:
    con.execute(f"""
    CREATE OR REPLACE VIEW v_doi AS
    SELECT *,
           lower(regexp_replace(regexp_replace(coalesce({q('doi')}, ''), '^https?://(dx\\.)?doi\\.org/', ''), '\\s+', '')) AS doi_norm
    FROM v
    """)
    dup_rows = con.execute(
        """
        SELECT COALESCE(SUM(c),0)::BIGINT FROM (
          SELECT COUNT(*) AS c
          FROM v_doi
          WHERE doi_norm <> ''
          GROUP BY doi_norm
          HAVING COUNT(*) > 1
        )
        """
    ).fetchone()[0]
    print("\nDOI_norm duplicate rows:", f"{dup_rows:,}")
    print("Top duplicated doi_norm:")
    print(con.execute(
        """
        SELECT doi_norm, COUNT(*) AS c
        FROM v_doi
        WHERE doi_norm <> ''
        GROUP BY 1
        HAVING COUNT(*) > 1
        ORDER BY c DESC
        LIMIT 10
        """
    ).fetch_df())
else:
    print("\n[doi] column not found; skip DOI checks.")

# 3) Year range/distribution (if year-like column exists)
year_col = None
for cand in ['year', 'pub_year', 'publication_year']:
    if cand in cols:
        year_col = cand
        break

if year_col:
    print("\nYear stats:")
    print(con.execute(
        f"""
        SELECT 
          MIN(try_cast({q(year_col)} AS INTEGER)) AS min_year,
          MAX(try_cast({q(year_col)} AS INTEGER)) AS max_year,
          SUM(CASE WHEN {q(year_col)} IS NULL THEN 1 ELSE 0 END) AS null_years
        FROM v
        """
    ).fetch_df())

    print("Out-of-range/unparsable years (should be small or zero):")
    print(con.execute(
        f"""
        SELECT COUNT(*) AS bad_years
        FROM v
        WHERE try_cast({q(year_col)} AS INTEGER) IS NULL
           OR try_cast({q(year_col)} AS INTEGER) < 2000
           OR try_cast({q(year_col)} AS INTEGER) > 2025
        """
    ).fetch_df())

    print("Counts by year:")
    print(con.execute(
        f"""
        SELECT try_cast({q(year_col)} AS INTEGER) AS y, COUNT(*) AS c
        FROM v
        GROUP BY 1
        ORDER BY 1
        """
    ).fetch_df())
else:
    print("\n[year] column not found; skip year checks.")

# 4) Key field quality (null rate, distinct counts)
key_fields = [c for c in ['title','authors','source','journal','year'] if c in cols]
if key_fields:
    print("\nKey field quality:")
    for c in key_fields:
        df = con.execute(
            f"""
            SELECT 
              COUNT(*)::BIGINT AS total,
              SUM(CASE WHEN {q(c)} IS NULL OR {q(c)}='' THEN 1 ELSE 0 END)::BIGINT AS nulls,
              COUNT(DISTINCT {q(c)})::BIGINT AS distinct_vals
            FROM v
            """
        ).fetch_df()
        df['null_ratio'] = (df['nulls'] / df['total']).round(4)
        print(f"[{c}]"); display(df)
else:
    print("\nNo key fields to profile.")

# 5) Optional: exact-duplicate rows across all columns (may be slower)
try:
    all_rows = con.execute("SELECT COUNT(*) FROM v").fetchone()[0]
    distinct_rows = con.execute("SELECT COUNT(*) FROM (SELECT DISTINCT * FROM v)").fetchone()[0]
    print("\nExact duplicate rows (all-columns):", f"{all_rows - distinct_rows:,}")
except Exception as e:
    print("\nExact-duplicate check skipped:", str(e))

# Sample a few rows
print("\nSample rows from Parquet:")
sample_df = pd.read_parquet(OUTPUT_PARQUET, engine='pyarrow').head(5)
sample_df


Counting CSV logical rows (this may take a moment)...
CSV logical rows: 358,493
Parquet rows: 358,493
Parquet columns: 76
Row groups: 3
✅ Row count validated: CSV and Parquet both have 358,493 records

[id] total=358,493, distinct=358,493, nulls=0, unique=True

DOI_norm duplicate rows: 118
Top duplicated doi_norm:
                               doi_norm  c
0          10.1299/jsmermd.2024.2a2-f08  2
1                  10.5937/zrffp45-7741  2
2         10.17762/turcomat.v12i1s.1764  2
3         10.1080/17512786.2022.2055620  2
4         10.31185/lark.vol2.iss45.2410  2
5             10.5294/pacla.2023.26.2.3  2
6                  10.2139/ssrn.3018521  2
7                10.3917/jibes.303.0103  2
8                     10.1093/ct/qtz004  2
9  10.22162/2587-6503-2022-1-21-134-173  2

Year stats:
   min_year  max_year  null_years
0      2000      2025         0.0
Out-of-range/unparsable years (should be small or zero):
   bad_years
0          2
Counts by year:
       y      c
0   2000   3620

,total,nulls,distinct_vals,null_ratio
0,358493,0,338123,0.0


[authors]


,total,nulls,distinct_vals,null_ratio
0,358493,18224,304988,0.0508


[year]


,total,nulls,distinct_vals,null_ratio
0,358493,0,28,0.0



Exact duplicate rows (all-columns): 0

Sample rows from Parquet:


,abstract,acknowledgements,altmetric,altmetric_id,arxiv_id,authors,authors_count,book_doi,book_series_title,book_title,...,score,source_title.id,source_title.title,subtitles,supporting_grant_ids,times_cited,title,type,volume,year
0,<p>Coverage of the Clinton-Lewinsky saga follo...,None,None,0.0,None,None,0,10.5771/9781461643852,None,Tabloid Tales,...,1.0,None,None,None,None,0,Preface,chapter,None,2000
1,<p>Coverage of the Clinton-Lewinsky saga follo...,None,None,0.0,None,None,0,10.5771/9781461643852,None,Tabloid Tales,...,1.0,None,None,None,None,0,Part One: Are the Tabloids Taking Over?,chapter,None,2000
2,<p>Coverage of the Clinton-Lewinsky saga follo...,None,None,0.0,None,None,0,10.5771/9781461643852,None,Tabloid Tales,...,1.0,None,None,None,None,0,About the Editors and Contributors,chapter,None,2000
3,<p>Coverage of the Clinton-Lewinsky saga follo...,None,None,0.0,None,None,0,10.5771/9781461643852,None,Tabloid Tales,...,1.0,None,None,None,None,0,Part Three: What Implications Does Tabloid Jou...,chapter,None,2000
4,<p>Coverage of the Clinton-Lewinsky saga follo...,None,None,0.0,None,None,0,10.5771/9781461643852,None,Tabloid Tales,...,1.0,None,None,None,None,0,Part Two: Tabloid Journalism in Perspective,chapter,None,2000


## Reproducibility Metadata
We record the versions of key libraries and basic runtime statistics to support reproducibility.


In [7]:
import platform
import sys

# Library versions
print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
print("pandas:", pd.__version__)
print("pyarrow:", pa.__version__)
print("duckdb:", duckdb.__version__)

# Basic file stats
if OUTPUT_CSV.exists():
    print("CSV size (MB):", round(OUTPUT_CSV.stat().st_size / (1024**2), 2))
if OUTPUT_PARQUET.exists():
    print("Parquet size (MB):", round(OUTPUT_PARQUET.stat().st_size / (1024**2), 2))


Python: 3.11.10
Platform: macOS-26.1-arm64-arm-64bit
pandas: 2.3.2
pyarrow: 21.0.0
duckdb: 1.4.0
CSV size (MB): 2268.29
Parquet size (MB): 695.48
